In [1]:
import os
os.chdir(r"C:\Users\GilseungAhn\Google 드라이브\작업\온라인 강의 머신러닝 성능 향상을 위한 데이터 탐색과 전처리 심화\데이터\실습 데이터 - #2")

import pandas as pd
import numpy as np
df = pd.read_excel("2018-20_코스피지수.xlsx", parse_date = True)

In [2]:
df.head()

,날짜,종가
0,2020-07-17,2201.19
1,2020-07-16,2183.76
2,2020-07-15,2201.88
3,2020-07-14,2183.61
4,2020-07-13,2186.06


In [3]:
# 순서를 고려하기 위해, 정렬이 필요
df.sort_values(by = ['날짜'], inplace = True) 

#### SAX 적용

In [4]:
def segmentation(x, w, a): # x: time series sample, w: window size, a: alphabet size
    window_mean = x[:w*int(len(x)/w)].reshape(w, -1).mean(axis = 1)
    if len(x) % w != 0:    
        window_mean = np.hstack([window_mean, x[w*int(len(x)/w):].mean()]) # 남겨진 부분은 그 부분만 평균해서 window_mean에 추가
    
    return window_mean

In [5]:
def find_break_points(wmv, a): # wmv:  window mean vector
    break_points = [np.quantile(wmv, (i+1)/a) for i in range(a)]
    return break_points

In [6]:
def conversion_window(wv, break_points): # wv: window mean vector 
    alphabet_sequence = []
    for e in wv:
        alphabet = 65
        for bp in break_points:
            if e < bp:
                alphabet_sequence.append(chr(alphabet))
                break
            else:
                alphabet += 1   
                
    return alphabet_sequence

In [7]:
KOSPI = df['종가'].values
window_mean_vector = segmentation(x = KOSPI, w = 100, a = 5)

In [8]:
window_mean_vector

array([2282.725     , 2284.32      , 2291.51      , 2293.9475    ,
       2273.465     , 2251.4475    , 2287.11      , 2310.595     ,
       2300.5325    , 2284.09      , 2304.1175    , 2331.6275    ,
       2316.5025    , 2219.9075    , 2154.9       , 2143.095     ,
       2046.02      , 2041.21      , 2084.465     , 2076.4525    ,
       2090.9       , 2071.75      , 2104.65      , 2104.07      ,
       2066.2725    , 2074.5325    , 2063.865     , 2026.8725    ,
       2016.58      , 2067.02      , 2108.655     , 2128.7975    ,
       2186.1475    , 2197.195     , 2199.6325    , 2210.5925    ,
       2229.58      , 2200.03      , 2154.2325    , 2159.345     ,
       2179.7725    , 2156.5575    , 2153.5575    , 2207.5025    ,
       2223.96      , 2237.7925    , 2213.585     , 2197.4575    ,
       2188.5175    , 2092.725     , 2067.995     , 2057.7525    ,
       2038.79      , 2061.4175    , 2098.095     , 2097.        ,
       2127.005     , 2127.1075    , 2114.1275    , 2071.3925 

In [9]:
bps = find_break_points(window_mean_vector, 5)
bps

[2064.98, 2090.9, 2154.2325, 2216.0199999999995, 2331.6275]

In [10]:
alphabet_sequence = conversion_window(window_mean_vector, bps)
alphabet_sequence

['E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'D',
 'C',
 'A',
 'A',
 'B',
 'B',
 'C',
 'B',
 'C',
 'C',
 'B',
 'B',
 'A',
 'A',
 'A',
 'B',
 'C',
 'C',
 'D',
 'D',
 'D',
 'D',
 'E',
 'D',
 'D',
 'D',
 'D',
 'D',
 'C',
 'D',
 'E',
 'E',
 'D',
 'D',
 'D',
 'C',
 'B',
 'A',
 'A',
 'A',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'B',
 'B',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'A',
 'A',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'B',
 'C',
 'D',
 'D',
 'D',
 'D',
 'E',
 'E',
 'D',
 'C',
 'E',
 'E',
 'D',
 'B',
 'A',
 'A']

#### 패턴 찾기

In [11]:
# 문자열로 변환 (하나의 리스트만 대상으로 하기에, 이렇게 하는 것이 더 수월. 단, L = 1으로만 한정)
alphabet_sequence = '-'.join(alphabet_sequence)
alphabet_sequence

'E-E-E-E-E-E-E-E-E-E-E-E-E-D-C-A-A-B-B-C-B-C-C-B-B-A-A-A-B-C-C-D-D-D-D-E-D-D-D-D-D-C-D-E-E-D-D-D-C-B-A-A-A-C-C-C-C-C-B-B-B-B-A-A-A-A-A-A-A-A-B-B-B-A-A-B-B-B-C-C-C-C-C-B-B-C-D-D-D-D-E-E-D-C-E-E-D-B-A-A'

In [12]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.01, L = 1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가시킴
    for item in item_set:
        # 문자열이므로 단순히 count method로 구현 가능
        occurence = sequence_data.count(item)
        if occurence / (sequence_data.count('-') + 1) >= min_support:
            queue.append(item)

    while queue:
        current_pattern = queue.pop() # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.count(current_pattern + '-' + item)
            if occurence / (sequence_data.count('-') + 1) >= min_support: # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False 
                queue.append(current_pattern + '-' + item)
        
        if check_maximum_frequent:
            maximum_frequent_sequence_item.append(current_pattern)        
    
    return maximum_frequent_sequence_item

In [13]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = {"부모":[], "자식":[], "지지도":[], "신뢰도":[]}
        
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산        
        co_occurence = sequence_data.count(sequence_item)
        support = co_occurence / (sequence_data.count('-') + 1)
        if co_occurence > min_support:
            for i in range(sequence_item.count('-') -1, 0, -1): # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = '-'.join(sequence_item.split('-')[:i])
                consequent = '-'.join(sequence_item.split('-')[i:])
                antecedent_occurence = sequence_data.count(antecedent)
               
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
            
    return pd.DataFrame(result)

In [14]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item([chr(65 + i) for i in range(5)], alphabet_sequence, min_support = 0.02, L = 1)

In [15]:
result = generate_association_rules(maximum_frequent_sequence_item, alphabet_sequence, min_support = 0.02, min_confidence = 0.1, L = 1)
result = result.sort_values(by = ['지지도', '신뢰도'], ascending = False)
result

,부모,신뢰도,자식,지지도
31,B-A,0.600000,A-A,0.03
22,C-C,0.500000,B-B,0.03
23,C,0.150000,C-B-B,0.03
24,C,0.150000,B-B,0.03
25,B,0.150000,C-C,0.03
26,B,0.150000,B-C,0.03
32,B,0.150000,A-A-A,0.03
10,C-D-D-D,1.000000,D-E,0.02
11,C-D-D,1.000000,D-D-E,0.02
13,C-C-C-C-C,1.000000,B-B,0.02
